<a href="https://colab.research.google.com/github/ludawg44/jigsawlabs/blob/master/29Apr20_7_coerce_to_boolean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coercing to Booleans

### Introduction

In this lesson, we'll work through identifying and coercing data to boolean values.  This will also prepare us to identify and coerce categorical values in our dataset.

### Loading our AirBnb Data

For this lesson, we'll work with [AirBnb listings in Berlin](https://www.kaggle.com/brittabettendorf/berlin-airbnb-data).  Let's load our data.

In [0]:
import pandas as pd
url = "https://raw.githubusercontent.com/jigsawlabs-student/engineering-large-datasets/master/nums_and_dates_ten_k.csv"
df = pd.read_csv(url, index_col = 0)

In [0]:
df.select_dtypes('object').shape

(8000, 50)

In [0]:
potential_date_cols = ['last_scraped',
 'host_since',
 'calendar_last_scraped',
 'first_review',
 'last_review']
df[potential_date_cols] = df[potential_date_cols].astype('datetime64')

In [0]:
df.shape

(8000, 83)

Lucky for us, we already have our good amount of our data already coerced.  But we still have more work to do.

In [0]:
object_df = df.select_dtypes(include = 'object')

object_df.shape

(8000, 45)

In [0]:
def contains_date(column):
#     remove nas first, potentially use all
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

### Feature engineering

So a lot of our columns are still of type object.  Let's take a look at some of our object columns. 

In [0]:
object_df.dtypes

listing_url                         object
name                                object
summary                             object
space                               object
description                         object
neighborhood_overview               object
notes                               object
transit                             object
access                              object
interaction                         object
house_rules                         object
picture_url                         object
host_url                            object
host_name                           object
host_location                       object
host_about                          object
host_response_time                  object
host_is_superhost                   object
host_thumbnail_url                  object
host_picture_url                    object
host_neighbourhood                  object
host_verifications                  object
host_has_profile_pic                object
host_identi

Where a larger percentage of the values in our columns repeat, we can think of them as categorical, and eventually one hot encode them.  So we wrote a method called `percent_different` that returns the percent of unique values that make up a series.  If most of the values in a series are unique, then it is not a categorical column.  

So in `find_categorical`, we loop through our columns, identifying those where `percent_different` is not too large - and those are our categorical columns.

In [0]:
def percent_different(df_series):
    series_filled = df_series.dropna()
    return len(series_filled.unique())/len(series_filled)

In [0]:
def find_categorical(df, threshold = .5):    
    categorical_df = pd.DataFrame({})
    for column in df.columns:
        if percent_different(df[column]) < threshold:
            categorical_df[column] = df[column]
    return categorical_df 

Let's see how this works.

In [0]:
potential_cat = find_categorical(object_df)

In [0]:
potential_cat.shape

(8000, 27)

In [0]:
potential_cat[:6]

,host_name,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,is_location_exact,property_type,room_type,bed_type,calendar_updated,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,Ian,"Key Biscayne, Florida, United States",within an hour,t,Mitte,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,Mitte,Berlin,Berlin,10119,Berlin,"Berlin, Germany",f,Guesthouse,Entire home/apt,Real Bed,3 months ago,t,f,strict_14_with_grace_period,f,f
1,Michael,"Berlin, Berlin, Germany",NaN,f,Prenzlauer Berg,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",NaN,Prenzlauer Berg Nordwest,Pankow,Berlin,Berlin,10437,Berlin,"Berlin, Germany",t,Apartment,Private room,Real Bed,7 weeks ago,t,f,flexible,f,f
2,Britta,"Coledale, New South Wales, Australia",within a day,f,Prenzlauer Berg,"['email', 'phone', 'facebook', 'reviews', 'man...",t,t,"Berlin, Berlin, Germany",Prenzlauer Berg,Prenzlauer Berg Südwest,Pankow,Berlin,Berlin,10405,Berlin,"Berlin, Germany",t,Apartment,Entire home/apt,Real Bed,a week ago,t,t,strict_14_with_grace_period,f,f
3,Jana,"Berlin, Berlin, Germany",NaN,f,Schöneberg,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",Schöneberg,Schöneberg-Nord,Tempelhof - Schöneberg,Berlin,Berlin,10777,Berlin,"Berlin, Germany",t,Apartment,Private room,Pull-out Sofa,4 weeks ago,t,f,strict_14_with_grace_period,f,f
4,Bright,"Berlin, Berlin, Germany",within an hour,t,Prenzlauer Berg,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",Prenzlauer Berg,Helmholtzplatz,Pankow,Berlin,Berlin,10437,Berlin,"Berlin, Germany",t,Apartment,Private room,Real Bed,3 days ago,t,f,moderate,f,f
5,Philipp,"Berlin, Berlin, Germany",within a few hours,f,Prenzlauer Berg,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Berlin, Berlin, Germany",Prenzlauer Berg,Prenzlauer Berg Südwest,Pankow,Berlin,Berlin,10405,Berlin,"Berlin, Germany",f,Apartment,Entire home/apt,Real Bed,2 weeks ago,t,f,strict_14_with_grace_period,f,f


It looks like it did a good job.

### Combine with Selecting Categorical Columns

The next step is to take a look at the values in those identified columns, to see if indeed they are full of categories.  Our `get_multiple_val_counts` method loops through a dataframe, providing the top `value_counts` values, and the related column.

In [0]:
def get_multiple_val_counts(df, num_vals = 1):
    return [df[column].value_counts(normalize=True).iloc[:num_vals] for column in df.columns]

In [0]:
get_multiple_val_counts(potential_cat)[:3]

[Anna    0.00852
 Name: host_name, dtype: float64, Berlin, Berlin, Germany    0.826202
 Name: host_location, dtype: float64, within an hour    0.457895
 Name: host_response_time, dtype: float64]

And summarize cats, puts this information in an easier to work with numpy array.

In [0]:
import numpy as np
def summarize_cats(df):
    multiple_val_counts = get_multiple_val_counts(df)
    stacked_counts = np.vstack([np.array([val_count.name, val_count.index[0], float(val_count.values[0])]) for val_count in multiple_val_counts])
    sorted_cols = np.argsort(stacked_counts.reshape(-1, 3)[:, 2].astype('float'))
    return stacked_counts[sorted_cols[::-1]]

In [0]:
summary = summarize_cats(potential_cat)
summary[:10]

array([['requires_license', 't', '0.99975'],
       ['market', 'Berlin', '0.99975'],
       ['state', 'Berlin', '0.9978704747588626'],
       ['host_has_profile_pic', 't', '0.9976193459466233'],
       ['smart_location', 'Berlin, Germany', '0.99175'],
       ['city', 'Berlin', '0.9917489686210776'],
       ['street', 'Berlin, Berlin, Germany', '0.989125'],
       ['require_guest_profile_picture', 'f', '0.984375'],
       ['require_guest_phone_verification', 'f', '0.975125'],
       ['bed_type', 'Real Bed', '0.93525']], dtype='<U55')

The first column in summary is the name of the column, the second is the top value, and the last column is the percent of the column the value was in. 

### Identifying Boolean Values

From the summary grid, we can start to see some strings that are really boolean values.  These are the columns with `t` or `f` as their top values.  

In [0]:
summary[:3]

array([['requires_license', 't', '0.99975'],
       ['market', 'Berlin', '0.99975'],
       ['state', 'Berlin', '0.9978704747588626']], dtype='<U55')

Let's select all of the columns from our summary that have values of `t` or `f`.

In [0]:
boolean_summary = summary[np.isin(summary[:, 1], ['t', 'f'])]

In [0]:
true_boolean_cols = boolean_summary[:, 0]
true_boolean_cols

array(['requires_license', 'host_has_profile_pic',
       'require_guest_profile_picture',
       'require_guest_phone_verification', 'host_is_superhost',
       'instant_bookable', 'is_location_exact', 'host_identity_verified'],
      dtype='<U55')

From here, we see that some of these columns have `na` values.

In [0]:
potential_cat[true_boolean_cols].isna().sum()/potential_cat.shape[0]

requires_license                    0.000000
host_has_profile_pic                0.002375
require_guest_profile_picture       0.000000
require_guest_phone_verification    0.000000
host_is_superhost                   0.002375
instant_bookable                    0.000000
is_location_exact                   0.000000
host_identity_verified              0.002375
dtype: float64

We'll have to coerce these values as well.  

> It's a small enough percentage that we can just coerce them to false values.

Now we'll use our MissingIndicator to convert these columns to have True or False values.  We do so, we by having the transformer set `t` to True, and all other values to False.  We'll also coerce the nan values to `f` beforehand.

> We can loop through to do this for each of our boolean columns.

In [0]:
from sklearn.impute import SimpleImputer, MissingIndicator
steps = [([col], [SimpleImputer(strategy = 'constant', missing_values= np.nan, fill_value = 'f'),
                  MissingIndicator(missing_values = 't')
                 ]
         ) 
         for col in true_boolean_cols]

And then place these steps in a DataFrameMapper.

In [0]:
from sklearn_pandas import DataFrameMapper
boolean_mapper = DataFrameMapper(steps, df_out = True)

In [0]:
#df[bools_df.columns]

In [0]:
bool_df = boolean_mapper.fit_transform(df)

In [0]:
bool_df[:2]

,requires_license,host_has_profile_pic,require_guest_profile_picture,require_guest_phone_verification,host_is_superhost,instant_bookable,is_location_exact,host_identity_verified
0,True,True,False,False,True,False,False,True
1,True,True,False,False,False,False,True,True


Then we can update our dataframe.

In [0]:
df.loc[:, bool_df.columns] = bool_df

In [0]:
df.select_dtypes('object').shape

(8000, 37)

And store our dataframe.

In [0]:
df.to_csv('./listings_coerced_booleans.csv')

And finally, we can store our datatypes.

In [0]:
import json
data = df.dtypes.astype(str).to_dict()

file = './coerced_bools_dtypes.json'

with open(file, 'w') as f:
    json.dump(data, f)

Then we can use numpy to identify our remaining potential_cat columns that we should coerce.

In [0]:
import numpy as np
remaining_cat_cols = np.setdiff1d(potential_cat.columns, bool_df.columns)

remaining_cat_cols

array(['bed_type', 'calendar_updated', 'cancellation_policy', 'city',
       'host_location', 'host_name', 'host_neighbourhood',
       'host_response_time', 'host_verifications', 'market',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'property_type', 'room_type',
       'smart_location', 'state', 'street', 'zipcode'], dtype=object)

### Summary

In this lesson, we were introduced to some of the methods for handling boolean and categorical data.  We saw that we identified our categorical columns by looking at the percent different.  If not a large percent of a column's values are different, it is likely categorical or boolean.  We then used our `summarize_cats` method to view the top values in each of the columns, along with how often they occur.

Finally, we used the `MissingImputer` to convert values in almost boolean columns to True and False values.